# Exploring Crawled Reddit Data

In [1]:
import json
import os
from pathlib import Path

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# spacy
import spacy

# sklearn
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
# pickle
import pickle

from typing import List, Dict, Tuple

# jupyter
from tqdm import tqdm
tqdm.pandas()

%load_ext autoreload
%autoreload 2

# we will be using utf-8 for io encoding decoding
os.environ['PYTHONUTF8'] = '1'

# global random state
global_random_state = 6471

In [2]:
Path.cwd()

WindowsPath('E:/gatech/courses/cs-6471-computational-social-science/project/utils/reddit')

In [3]:
data_dir = Path.cwd() / 'data'
crawled_data_dir = Path.cwd() / 'data/PushShiftAndRedditAPICrawler-output'
data_files = list(crawled_data_dir.glob('*.json'))

In [4]:
# load submissions and comments from json files
all_submissions = []
all_comments = []

for f in tqdm(data_files):
    f: Path
    data = json.load(f.open(mode='r', encoding='utf-8'))
    submission = data['submission']['data']

    # data['comments'] = {'type': 't1', 'data': 'actual comment data object'}
    comments = map(lambda c: c['data'], data['comments'])

    all_submissions.append(submission)
    all_comments.extend(comments)

100%|██████████| 20167/20167 [00:50<00:00, 399.63it/s] 


In [5]:
used_columns_df_submissions = [
    'author',
    'author_fullname',
    'created_utc',  # float64
    'name',  # text used as fullname
    'num_comments',  # int64
    'num_crossposts',  # int64
    'num_duplicates',  # int64
    'score',  # int64 num upvotes
    'selftext',  # longtext
    'subreddit_id',  # text fullname of subreddit
    'subreddit_name_prefixed',  # text prefixed name of the subreddit
    'subreddit_subscribers',  # 'int64'
    'subreddit_type',  # text
    'title',  # text
]
df_submissions = pd.DataFrame
df_submissions = pd.DataFrame(all_submissions)[used_columns_df_submissions]
df_submissions_created_utc = pd.to_datetime(df_submissions['created_utc'], unit='s', utc=True)
df_submissions['created_utc_year'] = df_submissions_created_utc.dt.year
df_submissions['created_utc_month'] = df_submissions_created_utc.dt.month
df_submissions['created_utc_day'] = df_submissions_created_utc.dt.day
df_submissions['created_utc_hour'] = df_submissions_created_utc.dt.hour
df_submissions['created_utc_minute'] = df_submissions_created_utc.dt.minute
df_submissions['created_utc_second'] = df_submissions_created_utc.dt.second

del df_submissions_created_utc
df_submissions

,author,author_fullname,created_utc,name,num_comments,num_crossposts,num_duplicates,score,selftext,subreddit_id,subreddit_name_prefixed,subreddit_subscribers,subreddit_type,title,created_utc_year,created_utc_month,created_utc_day,created_utc_hour,created_utc_minute,created_utc_second
0,[deleted],NaN,1.546344e+09,t3_abhq6i,2,0,0,2,[deleted],t5_2qhlc,r/privacy,1245595,public,Anyone tried Jami Messenger?,2019,1,1,12,6,20
1,amboris,t2_god9h,1.546349e+09,t3_abi6je,9,0,0,5,This post will bring me a lot of downvotes but...,t5_2tk0s,r/unpopularopinion,2383260,public,This is why i hate Discord and you should to! :/,2019,1,1,13,27,18
2,[deleted],NaN,1.546350e+09,t3_abiah9,43,0,0,221,[deleted],t5_2qh1q,r/india,765796,public,Ola's Privacy Policy is creepy,2019,1,1,13,46,36
3,EndMeetsEnd,t2_rln9c,1.546376e+09,t3_abltgc,0,0,1,1,,t5_skup3,r/TheLibertarianProject,2,public,Data Privacy Scandals and Public Policy Pickin...,2019,1,1,20,47,27
4,Armane407,t2_1alkf8uj,1.546376e+09,t3_ablxx1,1,0,0,1,\n\n￼\n\nLATEST NEWS\n\nFirewall Zero Hour Dev...,t5_9krdo,r/FireWallZeroHour,5447,public,Nice to know,2019,1,1,21,0,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20162,DeepAmoeba6593,t2_drk7ys57,1.640717e+09,t3_rqmklm,1,0,0,9,All accounts have lifetime warranty replacemen...,t5_yy3vb,r/realredditbay,2409,public,"[h] Over 10,500 orders and 740 feedback! [H] L...",2021,12,28,18,43,24
20163,DeepAmoeba6593,t2_drk7ys57,1.640717e+09,t3_rqml1e,1,0,0,6,All accounts have lifetime warranty replacemen...,t5_315l40,r/REDDITTBAY,2780,public,"[h] Over 10,500 orders and 740 feedback! [H] L...",2021,12,28,18,44,0
20164,DeepAmoeba6593,t2_drk7ys57,1.640717e+09,t3_rqmlo2,2,0,0,9,All accounts have lifetime warranty replacemen...,t5_2pn4ea,r/shoppingbay,27805,public,"[h] Over 10,500 orders and 740 feedback! [H] L...",2021,12,28,18,44,49
20165,DeepAmoeba6593,t2_drk7ys57,1.640717e+09,t3_rqmm6q,1,0,0,8,All accounts have lifetime warranty replacemen...,t5_25xkhi,r/Sales_and_Deals,1739,restricted,"[h] Over 10,500 orders and 740 feedback! [H] L...",2021,12,28,18,45,31


In [6]:
used_columns_df_comments = [
    'author',
    'author_fullname',
    'body',  # longtext content of the comment
    'controversiality',  # float64
    'created_utc',  # float64
    'depth',  # int
    'name',  # text fullname of the comment
    'parent_id',  # text fullname of the parent
    'score',  # int64
    'subreddit_id',  # text fullname of the subreddit
    'subreddit_name_prefixed',
]
df_comments = pd.DataFrame(all_comments)[used_columns_df_comments]
df_comments_created_utc = pd.to_datetime(df_comments['created_utc'], unit='s', utc=True)
df_comments['created_utc_year'] = df_comments_created_utc.dt.year
df_comments['created_utc_month'] = df_comments_created_utc.dt.month
df_comments['created_utc_day'] = df_comments_created_utc.dt.day
df_comments['created_utc_hour'] = df_comments_created_utc.dt.hour
df_comments['created_utc_minute'] = df_comments_created_utc.dt.minute
df_comments['created_utc_second'] = df_comments_created_utc.dt.second

del df_comments_created_utc
df_comments

,author,author_fullname,body,controversiality,created_utc,depth,name,parent_id,score,subreddit_id,subreddit_name_prefixed,created_utc_year,created_utc_month,created_utc_day,created_utc_hour,created_utc_minute,created_utc_second
0,[deleted],NaN,[deleted],0.0,1.546554e+09,0,t1_ed6jj48,t3_abhq6i,5.0,t5_2qhlc,r/privacy,2019.0,1.0,3.0,22.0,20.0,3.0
1,tedkotz,t2_hnvc9,They are not a service provider. That is proba...,0.0,1.554926e+09,0,t1_ekknevu,t3_abhq6i,1.0,t5_2qhlc,r/privacy,2019.0,4.0,10.0,19.0,58.0,17.0
2,AutoModerator,t2_6l4z3,Hi everyone! Please make sure to **upvote** we...,0.0,1.546349e+09,0,t1_ed0ev41,t3_abi6je,1.0,t5_2tk0s,r/unpopularopinion,2019.0,1.0,1.0,13.0,27.0,18.0
3,Flamingpanda2000,t2_1bd437bt,Nobody’s going to downvote your opinion becaus...,0.0,1.546350e+09,0,t1_ed0f60a,t3_abi6je,7.0,t5_2tk0s,r/unpopularopinion,2019.0,1.0,1.0,13.0,35.0,39.0
4,Vic9420,t2_mlbi9,"I use Discord,but only as a way to chat with a...",0.0,1.546350e+09,0,t1_ed0f89z,t3_abi6je,2.0,t5_2tk0s,r/unpopularopinion,2019.0,1.0,1.0,13.0,37.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65279,AutoModerator,t2_6l4z3,Hello /u/DeepAmoeba6593 for posting on r/shopp...,0.0,1.640717e+09,0,t1_hqb4o24,t3_rqmlo2,1.0,t5_2pn4ea,r/shoppingbay,2021.0,12.0,28.0,18.0,44.0,50.0
65280,No_Banana6319,t2_99xewa0x,https://discord.gg/xCUQd5ChrW,0.0,1.640718e+09,0,t1_hqb6h7s,t3_rqmmoe,1.0,t5_31ea4,r/TradeAccounts,2021.0,12.0,28.0,18.0,56.0,43.0
65281,allenbaha,t2_2l2656lf,Highly recommended!,0.0,1.640772e+09,0,t1_hqe6on0,t3_rqmmoe,1.0,t5_31ea4,r/TradeAccounts,2021.0,12.0,29.0,9.0,59.0,50.0
65282,flasm442,t2_2e893e6f,Don’t bother with this guy. He won’t refund yo...,0.0,1.640806e+09,0,t1_hqg36r4,t3_rqmmoe,1.0,t5_31ea4,r/TradeAccounts,2021.0,12.0,29.0,19.0,24.0,14.0


# Playground

## Submissions with valid selftext

In [7]:
mask_valid_selftext = (df_submissions['selftext'] != '') & (df_submissions['selftext'] != '[deleted]') & (df_submissions['selftext'] != '[removed]')
df_contented_submissions = df_submissions[mask_valid_selftext]
df_contented_submissions

,author,author_fullname,created_utc,name,num_comments,num_crossposts,num_duplicates,score,selftext,subreddit_id,subreddit_name_prefixed,subreddit_subscribers,subreddit_type,title,created_utc_year,created_utc_month,created_utc_day,created_utc_hour,created_utc_minute,created_utc_second
1,amboris,t2_god9h,1.546349e+09,t3_abi6je,9,0,0,5,This post will bring me a lot of downvotes but...,t5_2tk0s,r/unpopularopinion,2383260,public,This is why i hate Discord and you should to! :/,2019,1,1,13,27,18
4,Armane407,t2_1alkf8uj,1.546376e+09,t3_ablxx1,1,0,0,1,\n\n￼\n\nLATEST NEWS\n\nFirewall Zero Hour Dev...,t5_9krdo,r/FireWallZeroHour,5447,public,Nice to know,2019,1,1,21,0,48
6,grantph,t2_oimkt,1.546378e+09,t3_abm5w0,0,0,0,25,Let's narrow the focus of r/HailData to\n\n***...,t5_3nvqv,r/HailData,2420,public,The Future of HailData (2019),2019,1,1,21,24,7
8,avacash,t2_aee7w,1.546386e+09,t3_abnffw,40,0,0,3,"So, a bit of backstory: \n\nI am a violinist, ...",t5_2xhvq,r/AmItheAsshole,3742628,public,AITA for getting mad when people compliment my...,2019,1,1,23,42,9
9,dj-gutz,t2_hj8v9,1.546393e+09,t3_abog2j,0,0,0,0,[Cryptology ePrint Archive: Report 2018/415](h...,t5_se72n,r/myrXiv,16,public,Flux: Revisiting Near Blocks for Proof-of-Work...,2019,1,2,1,37,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20162,DeepAmoeba6593,t2_drk7ys57,1.640717e+09,t3_rqmklm,1,0,0,9,All accounts have lifetime warranty replacemen...,t5_yy3vb,r/realredditbay,2409,public,"[h] Over 10,500 orders and 740 feedback! [H] L...",2021,12,28,18,43,24
20163,DeepAmoeba6593,t2_drk7ys57,1.640717e+09,t3_rqml1e,1,0,0,6,All accounts have lifetime warranty replacemen...,t5_315l40,r/REDDITTBAY,2780,public,"[h] Over 10,500 orders and 740 feedback! [H] L...",2021,12,28,18,44,0
20164,DeepAmoeba6593,t2_drk7ys57,1.640717e+09,t3_rqmlo2,2,0,0,9,All accounts have lifetime warranty replacemen...,t5_2pn4ea,r/shoppingbay,27805,public,"[h] Over 10,500 orders and 740 feedback! [H] L...",2021,12,28,18,44,49
20165,DeepAmoeba6593,t2_drk7ys57,1.640717e+09,t3_rqmm6q,1,0,0,8,All accounts have lifetime warranty replacemen...,t5_25xkhi,r/Sales_and_Deals,1739,restricted,"[h] Over 10,500 orders and 740 feedback! [H] L...",2021,12,28,18,45,31


## Topic Modeling on Selftext with SpaCy

In [8]:
# initialize a spacy instance
nlp = spacy.load('en_core_web_sm')

In [9]:
processed_documents: List[spacy.language.Doc]
if not (data_dir / 'processed_documents.pkl').exists():
    processed_documents = []
    for doc in tqdm(nlp.pipe(
        df_contented_submissions['selftext'].tolist(),
            batch_size=32), total=len(df_contented_submissions)):
        processed_documents.append(doc)
    # save the processed documents
    processed_documents_save_file = data_dir / 'processed_documents.pkl'
    pickle.dump(processed_documents, processed_documents_save_file.open(mode='wb'))
else:
    processed_documents = pickle.load((data_dir / 'processed_documents.pkl').open(mode='rb'))

### LDA using Bag of Words

In [10]:
count_vectorizer = CountVectorizer(
    stop_words='english',
    ngram_range=(1, 3),
    max_df=0.9,
    min_df=3,
    vocabulary=nlp.vocab.strings
)

In [11]:
target_corpus = [' '.join([token.text.lower() for token in doc]) for doc in tqdm(processed_documents)]

100%|██████████| 12159/12159 [00:05<00:00, 2041.09it/s]


In [12]:
bow_processed_documents = count_vectorizer.fit_transform(target_corpus)

E:\Devs\Anaconda\envs\cs-37\lib\site-packages\sklearn\feature_extraction\text.py:1323: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  "Upper case characters found in"


In [13]:
lda = LatentDirichletAllocation(
    n_components=10,
    max_iter=20,
    batch_size=256,
    verbose=1,
    random_state=777
)

In [14]:
lda.fit(bow_processed_documents)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


LatentDirichletAllocation(batch_size=256, max_iter=20, random_state=777,
                          verbose=1)

In [15]:
lda_topics = lda.transform(bow_processed_documents)

In [16]:
lda_topics

array([[1.71795023e-01, 2.45124738e-04, 4.55308826e-02, ...,
        7.37915436e-02, 2.45142188e-04, 2.59281039e-02],
       [1.33771334e-01, 1.84535079e-04, 1.84547898e-04, ...,
        1.62801684e-01, 1.84562055e-04, 1.84541251e-04],
       [1.50240900e-01, 9.90343674e-04, 9.90345355e-04, ...,
        8.19526603e-02, 9.90348482e-04, 9.90321425e-04],
       ...,
       [2.62954485e-01, 1.69519479e-04, 1.77237290e-02, ...,
        1.69527672e-04, 1.69529023e-04, 6.90842111e-01],
       [2.62954485e-01, 1.69519479e-04, 1.77237290e-02, ...,
        1.69527672e-04, 1.69529023e-04, 6.90842111e-01],
       [2.62954485e-01, 1.69519479e-04, 1.77237290e-02, ...,
        1.69527672e-04, 1.69529023e-04, 6.90842111e-01]])

In [17]:
lda.components_

array([[0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       ...,
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1],
       [0.1, 0.1, 0.1, ..., 0.1, 0.1, 0.1]])

In [18]:
bow_processed_documents_words = count_vectorizer.get_feature_names()
for topic_id, topic in enumerate(lda.components_):
    print(f'Topic: {topic_id}: {" ".join([bow_processed_documents_words[i] for i in topic.argsort()[-1:-15:-1]])}')
    # break

Topic: 0: com account terms service free card money payment pay game credit 10 use website
Topic: 1: love government people world specialist state new states baba problem trump power public war
Topic: 2: support password email phone account users customer service technical mail app issues cash tech
Topic: 3: oddity shall use services terms law information rules en rights org right people including
Topic: 4: com www privacy data comments policy information use google user http security facebook app
Topic: 5: just like did time know people ve want really think way going good said
Topic: 6: edition com students png university ebook school student research pdf writing science health study
Topic: 7: company business com new www data market time product services marketing companies development high
Topic: 8: market service help customer report aol com user global att services free type point
Topic: 9: 99 month months com company 14 www companies india 11 pharmaceutical franchise points year


E:\Devs\Anaconda\envs\cs-37\lib\site-packages\sklearn\utils\deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [19]:
count_vectorizer.get_feature_names()

['\t',
 '\n',
 ' ',
 '  ',
 '!',
 '!!',
 '!!!',
 '!!!!',
 '!!!!!!!!!!!!!!!!',
 '!!!!.',
 '!!.',
 '!!?',
 '!!??',
 '!*',
 '!.',
 '!?',
 '!??',
 '"',
 '""',
 '#',
 "##'s",
 "##'x",
 "#'s",
 '#15',
 '#^%',
 '#dd',
 '$',
 '$19',
 '$Whose',
 '$Xxxxx',
 '$whose',
 '$xxxx',
 '%',
 '%-3',
 '%ach',
 '%ah',
 '%eh',
 '%er',
 '%ha',
 '%hm',
 '%huh',
 '%mm',
 '%oof',
 '%pw',
 '%uh',
 '%um',
 '%xx',
 '%xxx',
 '&',
 '&#',
 '&G.',
 '&L.',
 '&Ls',
 '&M.',
 '&P.',
 '&SA',
 '&T.',
 '&ex',
 '&in',
 '&ls',
 '&of',
 '&on',
 '&sa',
 '&the',
 '&to',
 '&uh',
 '&von',
 '&xx',
 '&xxx',
 "'",
 "''",
 "''It",
 "''Xx",
 "''it",
 "''xx",
 "'-(",
 "'-)",
 "'03",
 "'07",
 "'20s",
 "'30s",
 "'40s",
 "'45",
 "'46",
 "'50s",
 "'60s",
 "'67",
 "'68",
 "'69",
 "'70's",
 "'70s",
 "'71",
 "'73",
 "'74",
 "'76",
 "'78",
 "'80",
 "'80's",
 "'80s",
 "'82",
 "'86",
 "'89",
 "'90's",
 "'90s",
 "'91",
 "'94",
 "'96",
 "'97",
 "'98",
 "'99",
 "'Arabi",
 "'Cause",
 "'Connery",
 "'Cos",
 "'Coz",
 "'Cuz",
 "'Id",
 "'Il",
 "'It",
 "'N"

In [20]:
lda_10_topics_model = LatentDirichletAllocation(
    n_components=10,
    max_iter=20,
    batch_size=128,
    verbose=1,
    random_state=777,
)
lda_10_topics_model.fit(bow_processed_documents)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


LatentDirichletAllocation(max_iter=20, random_state=777, verbose=1)

In [21]:
lda_20_topics_model = LatentDirichletAllocation(
    n_components=10,
    max_iter=20,
    batch_size=128,
    verbose=1,
    random_state=777,
)
lda_20_topics_model.fit(bow_processed_documents)

iteration: 1 of max_iter: 20
iteration: 2 of max_iter: 20
iteration: 3 of max_iter: 20
iteration: 4 of max_iter: 20
iteration: 5 of max_iter: 20
iteration: 6 of max_iter: 20
iteration: 7 of max_iter: 20
iteration: 8 of max_iter: 20
iteration: 9 of max_iter: 20
iteration: 10 of max_iter: 20
iteration: 11 of max_iter: 20
iteration: 12 of max_iter: 20
iteration: 13 of max_iter: 20
iteration: 14 of max_iter: 20
iteration: 15 of max_iter: 20
iteration: 16 of max_iter: 20
iteration: 17 of max_iter: 20
iteration: 18 of max_iter: 20
iteration: 19 of max_iter: 20
iteration: 20 of max_iter: 20


LatentDirichletAllocation(max_iter=20, random_state=777, verbose=1)

In [22]:
lda_100_topics_model = LatentDirichletAllocation(
    n_components=100,
    max_iter=100,
    batch_size=128,
    verbose=2,
    random_state=777,
)
lda_100_topics_model.fit(bow_processed_documents)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   24.8s finished


iteration: 1 of max_iter: 100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   19.8s finished


iteration: 2 of max_iter: 100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   16.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


...

iteration: 99 of max_iter: 100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   12.6s finished


iteration: 100 of max_iter: 100


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    7.6s finished


LatentDirichletAllocation(max_iter=100, n_components=100, random_state=777,
                          verbose=2)

In [24]:
lda_1000_topics_model = LatentDirichletAllocation(
    n_components=1000,
    max_iter=300,
    batch_size=128,
    verbose=2,
    random_state=777,
)
lda_1000_topics_model.fit(bow_processed_documents)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.7min finished


iteration: 1 of max_iter: 300


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.9min finished


iteration: 2 of max_iter: 300


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.8min finished


...

iteration: 299 of max_iter: 300


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  1.7min finished


iteration: 300 of max_iter: 300


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   44.7s finished


LatentDirichletAllocation(max_iter=300, n_components=1000, random_state=777,
                          verbose=2)